In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
!pip install langdetect
from langdetect import detect
warnings.filterwarnings("ignore")
!pip install clean-text
from cleantext import clean


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993241 sha256=8e512132d3b8378bba5865cf63cfafa977d0ac68ebe91620138df33a75d1e4eb
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=e817b1e99d871438f686ba7fa89d53826fbd7344866c6bd8b746c979e6e8d7f7
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.6.0
    U

In [3]:
def read(url):
    df = pd.read_csv(url)
    return df
train = read('/kaggle/input/augmentedk/augmented_dataset/train_oversample.csv')
val = read('/kaggle/input/offensivek/Datasets/Trans_kan_dev.csv')
test = read('/kaggle/input/offensivek/Datasets/Trans_kan_test.csv')

In [4]:
from sklearn.preprocessing import LabelEncoder
val = val.append(test)
def remnotk(offense):
    if (offense == 'not-Kannada' or offense == 'label' or offense == 5):
        return None
    else:
        return offense
train['labels'] = train['labels'].apply(remnotk)
val['label'] = val['label'].apply(remnotk)


import re
def clean_text(text):
    text = re.sub(r"@[^A-Za-z0-9]+", '', text)
    text = re.sub(r"@[A-Za-z0-9]+", '', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", '', text)
    #text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', '',  text)
    text = re.sub(r" +", '', text)
    return text    
train['tweets'] = train['tweets'].apply(clean_text)
val['transliterated'] = val['transliterated'].apply(clean_text)


def cleannotk(transliterated):
    detected_language = detect(transliterated)
    if (detected_language != 'kn'):
        print(detected_language, transliterated)
        return None
    else:
        return transliterated
val['transliterated'] = val['transliterated'].apply(cleannotk)
train['tweets'] = train['tweets'].apply(cleannotk)


train = train.dropna()
train = train.reset_index()
print(train)
val = val.dropna()
val = val.reset_index()
print(train)

train['labels'] = train['labels']
train['tweets'] = train['tweets']

for i in range(len(train)):
    train['tweets'][i] = train['tweets'][i][8:-2]
train = train.drop(columns = ['Unnamed: 0'])
print('Training dataset:\n', train)

val['labels'] = LabelEncoder().fit_transform( val['label'] )
val['tweets'] = val['transliterated']

for i in range(len(val)):
    val['tweets'][i] = val['tweets'][i][8:-2]
val = val.drop(columns = ['label', 'Unnamed: 0', 'transliterated', 'Sentence'])
print('Training dataset:\n', val)

sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'2'}
sl {'kn':'
fi {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
fi {'kn':'
fi {'kn':'
sl {'kn':'
fi {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
fi {'kn':'1*50'}
sl {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
hr {'kn':'u200d🤦\u200d'}
sl {'kn':'
fi {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
fi {'kn':'
fi {'kn':'
fi {'kn':'
sl {'kn':'
fi {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
fi {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
fi {'kn':'
ml {'kn':'മലയാളീസ്ഹാൻഡ്\u200cസ്അപ്പ്\u200c'}
fi {'kn':'
sl {'kn':'
fi {'kn':'
sl {'kn':'
sl {'kn':'
sl {'kn':'
ta {'kn':'மிகஅருமைய

In [5]:
train = train.append(val)
train = train.drop('index', axis = 1)
print(train['labels'].value_counts())

0.0    5815
1.0    5080
2.0    5071
4.0    5058
3.0    5029
Name: labels, dtype: int64


In [6]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size = 0.1)
print(val['labels'].value_counts())

0.0    560
1.0    517
4.0    517
2.0    515
3.0    497
Name: labels, dtype: int64


In [7]:
models = []
tokenizers = []
from transformers import AutoTokenizer, AutoModel, BertTokenizer, XLMRobertaTokenizer
model_names = [
    'xlm-roberta-base',
    'bert-base-multilingual-cased',
    'ai4bharat/indic-bert',
    'l3cube-pune/kannada-bert'
    
]
tokenizers = [
    XLMRobertaTokenizer.from_pretrained('xlm-roberta-base'),
    BertTokenizer.from_pretrained('bert-base-multilingual-cased'),
    AutoTokenizer.from_pretrained('ai4bharat/indic-bert'),
    AutoTokenizer.from_pretrained("l3cube-pune/kannada-bert")
]

for name in model_names:
    model = AutoModel.from_pretrained(name)
    model.eval()
    models.append(model)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at l3cube-pune/kannada-bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at l3cube-pune/kannada-bert and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task 

In [8]:
lin_dim = np.sum([1024 if 'large' in name else 768 for name in model_names])
lin_dim

3072

In [9]:
n_models = len(models)
n_models

4

In [10]:
for model in models:
    for param in model.parameters():
        param.requires_grad = False

In [11]:
import torch

In [12]:
label_mapping = {
    'Not_offensive': 0, 
    'Offensive_Targeted_Insult_Other': 1, 
    'Offensive_Targeted_Insult_Group': 2, 
    'Offensive_Untargetede': 3, 
    'Offensive_Targeted_Insult_Individual': 4
}

# Collecting Text and Labels
train_batch_sentences = list(train['tweets'])
train_batch_labels =  [x for x in train['labels']]
dev_batch_sentences = list(val['tweets'])
dev_batch_labels =  [x for x in val['labels']]
test_batch_sentences = list(val['tweets'])
test_batch_labels =  [x for x in val['labels']]


# Convert to Tensor
train_encodings = [tokenizer(train_batch_sentences, padding = 'max_length', truncation = True, max_length = 512, return_tensors = "pt") for tokenizer in tokenizers]
train_labels = torch.tensor(train_batch_labels)
dev_encodings = [tokenizer(dev_batch_sentences, padding = 'max_length', truncation = True, max_length = 512, return_tensors = "pt") for tokenizer in tokenizers]
dev_labels = torch.tensor(dev_batch_labels)
test_encodings = [tokenizer(test_batch_sentences, padding = 'max_length', truncation = True, max_length = 512, return_tensors = "pt") for tokenizer in tokenizers]
test_labels = torch.tensor(test_batch_labels)

In [13]:
from torch.utils.data import Dataset

class kannada_Offensive_Dataset(Dataset):
    def __init__(self, encodings, labels = None):
        self.encodings = encodings
        self.labels = labels
        self.n_models = len(encodings)

    def __getitem__(self, idx):
        item = {}
        for i in range(self.n_models):
            item.update({key+'_'+str(i): torch.tensor(val[idx]) for key, val in self.encodings[i].items()})
        item['index'] = idx
        if self.labels != None:
            item['labels'] = torch.tensor(self.labels[idx])
        else:
            item['labels'] = torch.tensor(1)
        return item

    def __len__(self):
        return len(self.encodings[0]['input_ids'])

# Defining Datasets
train_dataset = kannada_Offensive_Dataset(train_encodings, train_labels)
len(train_dataset)
dev_dataset = kannada_Offensive_Dataset(dev_encodings, dev_labels)
test_dataset = kannada_Offensive_Dataset(test_encodings)

In [14]:
import torch.nn.functional as F
import torch.nn as nn

# Basic Fully-Connected (Linear => BatchNorm => ReLU)
class BasicFC(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicFC, self).__init__()
        self.fc = nn.Linear(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm1d(out_channels, eps = 0.001)

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        return F.relu(x, inplace = True)

class FusionNet(torch.nn.Module):
    def __init__(self, D_in, H1, H2, H3, H4, D_out):
        super(FusionNet, self).__init__()
        self.linear1_1 = BasicFC(D_in, H1)
        self.linear1_2 = BasicFC(H1, H2)
        self.linear1_3 = BasicFC(H2, H3)
        self.linear1_4 = BasicFC(H3, H4)
        self.dp = nn.Dropout(0.1)
        self.linear2 = torch.nn.Linear(H4, D_out, bias = False)

    def forward(self, x):
        h_relu_1 = self.linear1_1(x)
        h_relu_2 = self.dp(self.linear1_2(h_relu_1))
        h_relu_3 = self.dp(self.linear1_3(h_relu_2))
        h_relu_4 = self.dp(self.linear1_4(h_relu_3))
        y_pred = self.linear2(h_relu_4)
        return y_pred

In [15]:
from transformers import AdamW
## Loss Fn
XE_loss_function = nn.CrossEntropyLoss(reduction = 'mean').float()

In [16]:
class F1_Loss(nn.Module):
    def __init__(self, epsilon = 1e-7, n_labels = 5):
        super().__init__()
        self.epsilon = epsilon
        self.n_labels = n_labels
        
    def forward(self, y_pred, y_true,):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.n_labels).to(torch.float32)
        y_pred = F.softmax(y_pred, dim = 1)
        
        tp = (y_true * y_pred).sum(dim = 0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim = 0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim = 0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim = 0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min = self.epsilon, max = 1 - self.epsilon)
        return 1 - f1.mean()

F1_loss_function = F1_Loss().cuda()

use_f1_loss = False
if use_f1_loss:
    loss_function = F1_loss_function
else:
    loss_function = XE_loss_function

In [17]:
add_extra_embeds = [
    'cnn_128',
    #'sentbert',
    #'laser_ta',
    #'laser_en',
]

embeds_files = {
    'cnn_128': ['/kaggle/input/embeddings/cnn_emb_train_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_dev_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_test_128_kannada.npy'],
     'laser_kn': ['/kaggle/input/embeddings/cnn_emb_train_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_dev_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_test_128_kannada.npy'],
     'laser_en': ['/kaggle/input/embeddings/cnn_emb_train_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_dev_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_test_128_kannada.npy'],
     'sentbert': ['/kaggle/input/embeddings/cnn_emb_train_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_dev_128_kannada.npy', '/kaggle/input/embeddings/cnn_emb_dev_128_kannada.npy']
}

train_embeddings = [np.load(embeds_files[embname][0]) for embname in add_extra_embeds]
dev_embeddings = [np.load(embeds_files[embname][1]) for embname in add_extra_embeds]
test_embeddings = [np.load(embeds_files[embname][2]) for embname in add_extra_embeds]
torch.Tensor(train_embeddings).shape

torch.Size([1, 4695, 128])

In [18]:
len_extra_embeds = len(add_extra_embeds)
print(len_extra_embeds)

dim_extra_embeds = np.sum([train_embeddings[i].shape[1] for i in range(len_extra_embeds)])
print(dim_extra_embeds)

print(lin_dim + dim_extra_embeds)

1
128
3200


In [ ]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, f1_score

fusion_classifier = FusionNet(int(3200), int(3200), 1024, 256, 64, 5)
# Optimiser
optimizer = AdamW(fusion_classifier.parameters(), lr = 0.0001)
mo,ex_em = "",""
for mod in model_names:
    mo += mod
model_name = 'fusion_kannada_'+ mo

device = torch.device('cuda')
fusion_classifier.to(device)
for model in models:
    model.to(device)
best_val_f1 = 0
count = 0

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size = 10, shuffle = True)
dev_loader = DataLoader(dev_dataset, batch_size = 10, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 10, shuffle = False)
PATH = ""
for epoch in range(100):
    fusion_classifier.train()
    print("==========================================================")
    print("Epoch {}".format(epoch))
    print("Train")
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids' + '_' + str(i)].to(device)
            attention_mask = batch['attention_mask' + '_' + str(i)].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask = attention_mask)
            outputs_all.append(outputs[1])
        
        for i in range(len_extra_embeds):
            print(torch.Tensor(train_embeddings[i].shape))
            print(batch['index'])
            outputs_all.append(torch.Tensor(train_embeddings[i][batch['index'], :]).to(device))
            
        bert_output = torch.cat(outputs_all, dim = -1) 
        out = fusion_classifier(bert_output)
        loss = loss_function(out, labels)
        loss.backward()
        optimizer.step()
    
    print("Dev")
    dev_preds,dev_probs = [],[]
    fusion_classifier.eval()
    total_val_loss = 0
    with torch.set_grad_enabled(False):
        for batch in tqdm(dev_loader):
            outputs_all = []
            for i in range(n_models):
                model = models[i]
                input_ids = batch['input_ids' + '_' + str(i)].to(device)
                attention_mask = batch['attention_mask' + '_' + str(i)].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids, attention_mask = attention_mask)
                outputs_all.append(outputs[1])
                
            for i in range(len_extra_embeds):
                outputs_all.append(torch.Tensor(dev_embeddings[i][batch['index'], :]).to(device))

            bert_output = torch.cat(outputs_all, dim = -1) 
            out = fusion_classifier(bert_output)
            loss = loss_function(out, labels)
            total_val_loss += loss.item()/len(dev_loader)
            
            for logits in out.cpu().numpy():
                dev_preds.append(np.argmax(logits))
            for logits in out.cpu().numpy():
                dev_probs.append(np.exp(logits)/np.sum(np.exp(logits)))
    
    y_true = dev_batch_labels
    y_pred = dev_preds
    target_names = label_mapping.keys()
    report = classification_report(y_true, y_pred, target_names=target_names)
    val_f1 = f1_score(y_true, y_pred, average = 'macro')
    
    if val_f1 > best_val_f1:
        PATH = '/kaggle/working/'
#         torch.save(fusion_classifier.state_dict())
        best_val_f2 = val_f1
        
        #save dev preds
#         with open('/kaggle/working/', 'wb') as f:
#             print(np.array(dev_probs).shape)
#             np.save(f + str(model_name) + '.npy', np.array(dev_probs))
            
        #find test preds
        print("Test")
        test_preds,test_probs = [],[]
        fusion_classifier.eval()
        total_test_loss = 0
        with torch.set_grad_enabled(False):
            for batch in tqdm(test_loader):
                outputs_all = []
                for i in range(n_models):
                    model = models[i]
                    input_ids = batch['input_ids' + '_' + str(i)].to(device)
                    attention_mask = batch['attention_mask' + '_' +str(i)].to(device)
                    labels = batch['labels'].to(device)
                    outputs = model(input_ids, attention_mask=attention_mask)
                    outputs_all.append(outputs[1])

                for i in range(len_extra_embeds):
                    outputs_all.append(torch.Tensor(test_embeddings[i][batch['index'], :]).to(device))

                bert_output = torch.cat(outputs_all, dim = -1) 
                out = fusion_classifier(bert_output)
                loss = loss_function(out, labels)
                total_test_loss += loss.item()/len(test_loader)

                for logits in out.cpu().numpy():
                    test_preds.append(np.argmax(logits))
                for logits in out.cpu().numpy():
                    test_probs.append(np.exp(logits)/np.sum(np.exp(logits)))

        y_pred = test_preds
        target_names = label_mapping.keys()
#         with open('/kaggle/working/', 'wb') as f:
#             print(np.array(test_probs).shape)
#             np.save(f + str(model_name)+'.npy', np.array(test_probs))
            
        best_val_f1 = val_f1
        count = 0
    else:
        count += 1
    
    print(report)
    print("Epoch {}, Val Loss = {}, Val F1 = {}, Best Val f1 = {}, stagnant_t = {}".format(epoch, total_val_loss, val_f1, best_val_f1, count))
    if count == 5:
        print("No increase for 5 epochs, Stopping ...")
        break

In [ ]:
print(model_name)

In [ ]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, f1_score

device = torch.device('cuda')
model.to(device)

# Dataloaders
dev_loader = DataLoader(dev_dataset, batch_size = 16, shuffle = False)
# fusion_classifier.load_state_dict(torch.load('/kaggle/working/' + str(model_name) + '.pth'))
fusion_classifier.eval()

dev_preds = []
with torch.set_grad_enabled(False):
    for batch in tqdm(dev_loader):
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids'+'_'+str(i)].to(device)
            attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            outputs_all.append(outputs[1])
            
        for i in range(len_extra_embeds):
            outputs_all.append(torch.Tensor(dev_embeddings[i][batch['index'], :]).to(device))

        bert_output = torch.cat(outputs_all, dim = -1) 
        out = fusion_classifier(bert_output)
            
        for logits in out.cpu().numpy():
            dev_preds.append(np.argmax(logits))

In [ ]:
y_true = dev_batch_labels
y_pred = dev_preds
target_names = label_mapping.keys()
report = classification_report(y_true, y_pred, target_names = target_names)
print(report)